In [2]:
import pandas as pd

In [3]:
re_dust_df = pd.read_csv('../data/seoul_re_dust.csv', index_col=0)
living_df = pd.read_csv('../data/living_df.csv', index_col=0)
fine_dust_df = pd.read_csv('../data/fine_dust_df.csv', index_col=0)

In [4]:
re_dust_df.head(3)

,date,time,city,gu,road,start,end,temperature,humidity,avg_re_dust,dust_ratio
0,2018-05-03,09:50:00,서울,강서구,개화동로,서울특별시 강서구 개화동 659천,서울특별시 강서구 방화동 886-15 도,15.0,46.0,5.0,매우좋음
1,2018-05-03,10:13:00,서울,강서구,공항대로,서울특별시 강서구 방화동 621-4도,서울특별시 양천구 목동 560도,15.0,45.0,20.0,매우좋음
2,2018-05-03,10:41:00,서울,강서구,양천로,서울특별시 강서구 염창동 259-8제,서울특별시 강서구 방화동 515-4도,15.0,47.0,39.0,매우좋음


In [5]:
re_dust_df['date_cd'] = re_dust_df['date'].apply(lambda x: "".join(x.split('-')))

In [6]:
main_df = re_dust_df.groupby(['date_cd', 'gu'])[['temperature', 'humidity', 'avg_re_dust']].mean().reset_index()

In [7]:
fine_dust_df.head(3)

,MSRDT_DE,MSRRGN_NM,MSRSTE_NM,PM10,PM25,O3,NO2,CO,SO2
0,20180503,도심권,중구,22,11,0.036,0.022,0.4,0.002
1,20180503,도심권,종로구,23,11,0.039,0.021,0.5,0.005
2,20180503,도심권,용산구,23,12,0.031,0.024,0.4,0.002


In [8]:
fine_dust_df['MSRDT_DE'] = fine_dust_df['MSRDT_DE'].astype(str)

In [9]:
merged_df = pd.merge(main_df, 
                     fine_dust_df, 
                     left_on=["date_cd", "gu"], 
                     right_on=["MSRDT_DE", "MSRSTE_NM"])

In [10]:
map_df = pd.read_csv('../data_preprocess/행정동코드_매핑정보_20200325.csv')
map_df.head(3)

,RESD_CD,RESD_DO_NM,RESC_CT_NM
0,11110,서울,종로구
1,11140,서울,중구
2,11170,서울,용산구


In [11]:
living_df = pd.merge(living_df, map_df, left_on='ADSTRD_CODE_SE', right_on='RESD_CD')

In [12]:
living_df.head(3)

,STDR_DE_ID,ADSTRD_CODE_SE,TOT_LVPOP_CO,RESD_CD,RESD_DO_NM,RESC_CT_NM
0,20180327,11110,366210.103583,11110,서울,종로구
1,20180328,11110,370298.500154,11110,서울,종로구
2,20180329,11110,367064.571029,11110,서울,종로구


In [13]:
living_df['STDR_DE_ID'] = living_df['STDR_DE_ID'].astype(str)

In [14]:
df = pd.merge(merged_df, 
              living_df, 
              left_on=["date_cd", "gu"],
              right_on=["STDR_DE_ID", "RESC_CT_NM"])

In [15]:
columns = [
    'date_cd', 'gu', 'temperature', 'humidity', 'avg_re_dust', 
    'PM10', 'PM25', 'O3', 'NO2', 'CO', 'SO2', 'TOT_LVPOP_CO'
]

In [16]:
df = df[columns]

In [17]:
df.to_csv('../data/main_df.csv')